In [9]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [20]:
%run -i ../../core.py

In [21]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [22]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.tensor(np.median(dummy_data))
# print(int(len(dummy_data)/2))
dummy_lower_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)-1])
dummy_upper_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)])


In [23]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = dummy_lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = dummy_upper_to_median, requires_grad = False)
    def forward(self,X):
        count_less = torch.sum((X < self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))

        # doesnt support mod if not in conditional
       
        # For count_equal > 0
        if(len%2==0):
            is_odd = 0
        else:
            is_odd = 1
        less_cons = count_less<half_len+is_odd
        more_cons = count_less+count_equal>half_len


        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<self.upper).double())==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
        return (median_in_cons*(count_equal!=0)+median_out_cons*(count_equal ==0), self.w)

    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

dummy output:  tensor(6.1500, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_55637/4127999846.py:19: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if(len%2==0):


In [24]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(np.median(data))
lower_to_median = torch.tensor(np.sort(data)[int(len(data)/2)-1])
upper_to_median = torch.tensor(np.sort(data)[int(len(data)/2)])
print("Theory_output: ", theory_output)
print("lower: ", lower_to_median)
print("upper: ", upper_to_median)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = upper_to_median, requires_grad = False)
    def forward(self,X):
        count_less = torch.sum((X < self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))

        # doesnt support mod if not in conditional
       
        # For count_equal > 0
        if(len%2==0):
            is_odd = 0
        else:
            is_odd = 1
        less_cons = count_less<half_len+is_odd
        more_cons = count_less+count_equal>half_len


        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<self.upper).double())==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
        return (median_in_cons*(count_equal!=0)+median_out_cons*(count_equal ==0), self.w)

 

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory_output:  tensor(4.6500, dtype=torch.float64)
lower:  tensor(4.1000, dtype=torch.float64)
upper:  tensor(5.2000, dtype=torch.float64)
==== Generate & Calibrate Setting ====


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_55637/2517425331.py:25: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if(len%2==0):


scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":10,"lookup_range":[-790,826],"logrows":11,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":1312,"total_assignments":287,"total_const_size":55,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,5],"model_input_scales":[5],"module_sizes":{"kzg":[],"poseidon":[1312,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [25]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 0.3089122772216797 seconds
Theory output:  tensor(4.6500, dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 4.65625
==== Generating Proof ====
proof:  {'instances': [[[15776967246226738966, 14470885921245171519, 5947983710744493689, 2326010236508186386], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [898490808456445164, 4627731094189474011, 12171376262565855552, 810232741982306196]]], 'proof': '10cfad4b5f97ef0f52fa51ff1aa68257108311873180b5f101f17fe03cedad950bb05c03f35f36b593e9df3697024ca6a884f73b3c6668ca923570a8bbfb323715178cf6867390fb62706be40159d98768f18d993b59fa6bc870a914216e57eb2010387e1cf38271c26eecc7bb3d27f9196d08e6bb1fa37d206170e4bb82eed822ba6239d6c9262988460c57febfacb2af0cbd99de40b9ac12086cc7c17fa70d0488c520011fc2f5452c40facd164ebaec5a091d79002052ef873a874d1f768205d4a43fa581b979b6049befbafe3053576873324593f9fa8a9adef5dd3784d01d45f8b9a2bcd03a7f19363916509859fc7eaa22a0ee3e632aaa2d1885a8

In [26]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[15776967246226738966, 14470885921245171519, 5947983710744493689, 2326010236508186386], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [898490808456445164, 4627731094189474011, 12171376262565855552, 810232741982306196]]]
proof boolean:  1.0
proof result 1 : 4.65625
verified
